In [ ]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from torchvision import transforms
from tqdm.notebook import tqdm

In [ ]:
!pipo install kaggle

/bin/bash: line 1: pipo: command not found


In [ ]:
from google.colab import files
from google.colab import drive
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohssine264","key":"5a9634eefaa01bbe07e6ae29525aaf5e"}'}

In [ ]:
!mdkir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

/bin/bash: line 1: mdkir: command not found
cp: cannot create regular file '/root/.kaggle/': Not a directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
drive.mount("/content/drive")

ValueError: mount failed

In [ ]:
!kaggle datasets download -d vanshikavmittal/fakeddit-dataset

In [ ]:
!unzip fakeddit-dataset.zip -d fakeddit-dataset

In [ ]:
import os
dataset_path = "/content/multimodal_only_samples"
print("Path to dataset files:" , os.listdir(dataset_path))

In [ ]:
import glob
tsv_files = glob.glob(os.path.join(dataset_path, "*.tsv"))
print("Number of tsv files:" , len(tsv_files))

In [ ]:
import pandas as pd

DATASET_DIR = "/content/multimodal_only_samples"

train_path = os.path.join(DATASET_DIR, "multimodal_train.tsv")
train_df = pd.read_csv(train_path, sep="\t")
print("Training set sample rows:")
print(train_df.head())
print("Training set columns:", train_df.columns.tolist())
print("Number of training samples:",len(train_df))

In [ ]:
!unzip fakeddit-dataset.zip -d /content/

In [ ]:
val_path = os.path.join(DATASET_DIR,"multimodal_validate.tsv")
val_df = pd.read_csv(val_path, sep="\t")
print("Validation set sample rows:")
print(val_df.head())
print("Validation set columns:", val_df.columns.tolist())
print("Number of validation samples" , len(val_df))

In [ ]:
test_path = os.path.join(DATASET_DIR, "multimodal_test_public.tsv")
test_df = pd.read_csv(test_path, sep="\t")
print("\nTest set sample rows:")
print(test_df.head())
print("Test set columns:" , test_df.columns.tolist())
print("Number of test samples:" , len(test_df))

In [ ]:
train_df

In [ ]:
#fixing the time
train_df["created_utc"] = pd.to_datetime(train_df["created_utc"], unit="s")

In [ ]:
#cleaning the text
train_df["clean_title"] = train_df["clean_title"].str.strip().str.lower()
train_df["title"] = train_df["title"].str.strip().str.lower()

In [ ]:
#missing values
train_df["linked_submission_id"] = train_df["linked_submission_id"].fillna("")

In [ ]:
# Convert numeric columns ensuring proper type
train_df["num_comments"] = pd.to_numeric(train_df["num_comments"] , errors="coerce")
train_df["score"] = pd.to_numeric(train_df["score"], errors="coerce")
train_df["upvote_ratio"] = pd.to_numeric(train_df["upvote_ratio"], errors="coerce")

In [ ]:

df = train_df[~((train_df["hasImage"] == True )& (train_df["image_url"].isna()))]

In [ ]:
df.drop_duplicates(subset=["id"],inplace=True)

In [ ]:
print("Cleaned DataFrame shape:", df.shape)
print(df.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x="2_way_label", data= df)
plt.title("Distribution of 2-way labels(Fake vs. Real)")
plt.xlabel("2-way label (0=Fake, 1=Real)")
plt.ylabel("Count")
plt.show()

In [ ]:
df['year'] = df['created_utc'].dt.year
plt.figure(figsize=(10,6))
sns.countplot(x='year', data=df, palette="viridis")
plt.title("Number of Posts per Year")
plt.xlabel("Year")
plt.ylabel("Count")
plt.show()



In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='hasImage', data=df)
plt.title("Distribution of hasImage")
plt.xlabel("hasImage (True/False)")
plt.ylabel("Count")
plt.show()

In [ ]:
from PIL import Image
import requests
from io import BytesIO
from torchvision import transforms



In [ ]:
# Define the transformation pipeline
image_transforms = transforms.Compose([
    transforms.Resize((256, 256)),      # Resize image to 256x256
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
def load_and_preprocess_image(image_path_or_url):
  try:
    #Checking if the input is a url or a laocal file path
    if image_path_or_url.startswith("http"):
      response = requests.get(image_path_or_url)
      img = Image.open(BytesIO(response.content)).convert("RGB")
    else:
      img = Image.open(image_path_or_url).convert("RGB")
  except Exception as e:
    print(f"Error loading image: {e}")
    return None


In [ ]:
sample_image_url =

In [ ]:
df["image_url"][0]

In [ ]:
df = df[df["hasImage"] == True].reset_index(drop=True)


In [ ]:
tqdm.pandas()
df['processed_image'] = df.progress_apply(
    lambda row: load_and_preprocess_image(row['image_url']) if pd.notnull(row['image_url']) else None,
    axis=1
)


In [ ]:
print("Number of processed images:", df['processed_image'].notna().sum())
print(df[['id', 'image_url', 'processed_image']].head())

In [ ]:
sample_image_url = "https://external-preview.redd.it/WylDbZrnbvZdBpgfa3ntxYf17CBHndiJWHylVm2j_nY.jpg?width=320&crop=smart&auto=webp&s=449659a10792de4d55c2f27d2176fdc8bc66e72a"
img_tensor = load_and_preprocess_image(sample_image_url)

In [ ]:
if img_tensor is not None:
  print("Image tensor shape:", img_tensor.shape)
else:
  print("Failed to load or preprocess the image.")